<a href="https://colab.research.google.com/github/jamalromero/colab/blob/main/init_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%loadFromPOM
<dependency>
    <groupId>tech.tablesaw</groupId>
    <artifactId>tablesaw-core</artifactId>
    <version>0.43.1</version>
</dependency>
<dependency>
  <groupId>com.github.haifengl</groupId>
  <artifactId>smile-plot</artifactId>
  <version>3.1.1</version>
</dependency>
<dependency>
    <groupId>ai.djl</groupId>
    <artifactId>api</artifactId>
    <version>0.29.0</version>
</dependency>
<dependency>
    <groupId>ai.djl.pytorch</groupId>
    <artifactId>pytorch-engine</artifactId>
    <version>0.29.0</version>
</dependency>
<dependency>
  <groupId>org.slf4j</groupId>
  <artifactId>slf4j-simple</artifactId>
  <version>2.0.13</version>
</dependency>
<dependency>
  <groupId>org.slf4j</groupId>
  <artifactId>slf4j-api</artifactId>
  <version>2.0.13</version>
</dependency>
<dependency>
    <groupId>commons-io</groupId>
    <artifactId>commons-io</artifactId>
    <version>2.16.1</version>
</dependency>

In [ ]:
import java.util.*;
import java.util.stream.*;
import java.awt.Graphics2D;
import java.awt.Color;
import java.awt.image.BufferedImage;

import ai.djl.*;
import ai.djl.engine.*;
import ai.djl.inference.*;
import ai.djl.modality.*;
import ai.djl.modality.cv.*;
import ai.djl.modality.cv.Image;
import ai.djl.modality.cv.transform.*;
import ai.djl.modality.cv.translator.*;
import ai.djl.modality.cv.output.*;
import ai.djl.modality.cv.util.*;
import ai.djl.ndarray.*;
import ai.djl.ndarray.index.*;
import ai.djl.ndarray.types.*;
import ai.djl.ndarray.types.Shape;
import ai.djl.nn.*;
import ai.djl.nn.convolutional.*;
import ai.djl.nn.core.*;
import ai.djl.nn.norm.*;
import ai.djl.nn.pooling.*;
import ai.djl.nn.recurrent.*;
import ai.djl.repository.zoo.*;
import ai.djl.training.*;
import ai.djl.training.evaluator.*;
import ai.djl.training.initializer.*;
import ai.djl.training.listener.*;
import ai.djl.training.loss.*;
import ai.djl.training.optimizer.*;
import ai.djl.training.tracker.*;
import ai.djl.training.util.*;
import ai.djl.training.dataset.*;
import ai.djl.translate.*;
import ai.djl.util.*;
import java.io.*;
import java.net.*;
import java.nio.*;
import java.nio.file.*;
import java.util.*;
import java.util.function.*;

import tech.tablesaw.api.*;
import tech.tablesaw.columns.*;
import tech.tablesaw.io.csv.*;
import tech.tablesaw.aggregate.AggregateFunctions;
import tech.tablesaw.conversion.*;

import smile.plot.swing.*;

import org.apache.commons.math3.util.*;

In [ ]:
double[][] toDoubleArray(NDArray ndArray) {
  List<double[]> list = new ArrayList<>();
  for (int i = 0; i < ndArray.getShape().get(0); i++) {
    list.add(ndArray.get(i).toDoubleArray());
  }
  double[][] array = new double[list.size()][];
  for (int i = 0; i < list.size(); i++) {
    array[i] = list.get(i);
  }
  return array;
}

In [ ]:
HashMap<String, NDArray> meshgrid(NDArray x1, NDArray x2) {
    var x1_mesh = x1.reshape(1, -1).broadcast(x2.size(), -1);
    var x2_mesh = x2.reshape(-1, 1).broadcast(-1, x1.size());
    HashMap<String, NDArray> meshMap = new HashMap<>();
    meshMap.put("x1", x1_mesh);
    meshMap.put("x2", x2_mesh);
    return meshMap;
}

In [ ]:
BufferedImage plotGrid(List<Canvas> canvases, int cols) {
    int tile_side = 300;
    int rows = canvases.size() % cols == 0? canvases.size() / cols: (int)Math.floor(canvases.size() / cols) + 1;
    BufferedImage combinedImage = new BufferedImage(tile_side * cols, tile_side * rows, BufferedImage.TYPE_INT_ARGB);
    Graphics2D g2d = combinedImage.createGraphics();
    for(int i = 0; i <  canvases.size(); i++) {
      int tile_x = tile_side * (i % cols);
      int tile_y = tile_side * ((i / cols) % rows);
      g2d.drawImage(canvases.get(i).toBufferedImage(tile_side, tile_side), tile_x, tile_y, null);
    }
    g2d.dispose();
    return combinedImage;
}